In [51]:
# Querying Yahoo Finance Data

In [1]:
import requests
import pandas as pd
import numpy as np

# Data Retrieval

In [2]:
# Captures the Equity page to retrieve Expiration Dates

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

ticker = 'SPY'

req = requests.get('http://query1.finance.yahoo.com/v7/finance/options/' + ticker, headers = headers)

In [3]:
# Decode the response
dates = pd.read_json(req.content.decode())

# Translate Unix Timestamps into Python TimeDate 
dates_unix = dates.optionChain[1][0]['expirationDates']
dates_timedate = pd.to_datetime(dates_unix, origin='unix', unit = 's')

# Convert to Human-readable form
dates_human = []
for i, each in enumerate(dates_timedate):
    dates_human.append(str(each.year) + '-' + str(each.month) + '-' + str(each.day))


In [27]:
dates_unix

[1648425600,
 1648598400,
 1648684800,
 1648771200,
 1649030400,
 1649203200,
 1649376000,
 1649635200,
 1649808000,
 1649894400,
 1650240000,
 1650412800,
 1650585600,
 1650844800,
 1651190400,
 1653004800,
 1655424000,
 1656547200,
 1657843200,
 1660867200,
 1663286400,
 1664496000,
 1671148800,
 1672358400,
 1674172800,
 1679011200,
 1686873600,
 1702598400,
 1705622400,
 1734652800]

In [35]:
import datetime

In [44]:
datetime.datetime.today().strftime('%s')

'1648265329'

In [47]:
print( (float(dates_unix[1]) - float(datetime.datetime.today().strftime('%s'))) / (60*60*24) )

3.854513888888889


In [24]:
dates.optionChain[1][0]['quote']['regularMarketPrice']

452.69

In [67]:
# Request Option chains for each corresponding Expiration Date (Human-Readable)

all_exp = {}
for i, exp_date in enumerate(dates_unix):
    all_exp[dates_human[i]] = pd.read_json(requests.get('http://query1.finance.yahoo.com/v7/finance/options/' + ticker + '?date=' + str(exp_date), headers = headers).content.decode())

In [68]:
# Assure the dictionary contains all Expiration Dates of interest
all_exp.keys()

dict_keys(['2022-2-4', '2022-2-11', '2022-2-18', '2022-2-25', '2022-3-4', '2022-3-18', '2022-4-14', '2022-5-20', '2022-6-17', '2022-7-15', '2022-9-16', '2023-1-20', '2023-5-19', '2023-6-16', '2024-1-19'])

In [69]:
# Convert the format from Dictionaties to Pandas Dataframe
# Each Chain can now be accessed by quering 'YY-MM-DD' Expiration date in the Dictionary

for each in all_exp:
    all_exp[each] = pd.DataFrame.from_dict(all_exp[each].optionChain.result[0], orient='index')

In [70]:
all_exp['2023-3-17']

KeyError: '2023-3-17'

# Datapoints of Interest

## Equity

In [ ]:
# Information Accessible for a Single Equity
all_exp['2023-3-17'][0][4].keys()

dict_keys(['language', 'region', 'quoteType', 'quoteSourceName', 'triggerable', 'currency', 'exchange', 'shortName', 'longName', 'messageBoardId', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'market', 'esgPopulated', 'sharesOutstanding', 'bookValue', 'fiftyDayAverage', 'fiftyDayAverageChange', 'fiftyDayAverageChangePercent', 'twoHundredDayAverage', 'twoHundredDayAverageChange', 'twoHundredDayAverageChangePercent', 'marketCap', 'priceToBook', 'sourceInterval', 'exchangeDataDelayedBy', 'tradeable', 'postMarketChangePercent', 'postMarketTime', 'postMarketPrice', 'postMarketChange', 'regularMarketChange', 'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice', 'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow', 'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask', 'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency', 'regularMarketOpen', 'averageDailyVolume3Month', 'averageDailyVolume10Day', 'fifty

In [ ]:
# All Expiration Dates available in UNIX Timestamp format
all_exp['2023-3-17'][0]['expirationDates']

[1643587200,
 1643760000,
 1643932800,
 1644192000,
 1644364800,
 1644537600,
 1644796800,
 1644969600,
 1645142400,
 1645488000,
 1645574400,
 1645747200,
 1646006400,
 1646352000,
 1647561600,
 1648684800,
 1649894400,
 1653004800,
 1655424000,
 1656547200,
 1663286400,
 1664496000,
 1671148800,
 1672358400,
 1674172800,
 1679011200,
 1686873600,
 1702598400,
 1705622400,
 1734652800]

## Option Chain

In [ ]:
# Datapoints available for Each Contract
all_exp['2023-3-17'].loc['options'][0][0]['calls'][0].keys()

dict_keys(['contractSymbol', 'strike', 'currency', 'lastPrice', 'change', 'percentChange', 'volume', 'openInterest', 'bid', 'ask', 'contractSize', 'expiration', 'lastTradeDate', 'impliedVolatility', 'inTheMoney'])

In [ ]:
# Current Expiration Date
all_exp['2023-3-17'].loc['options'][0][0]['expirationDate']

# All Puts and Calls in order (Calls & Puts: Low to High Strike)
all_exp['2023-3-17'].loc['options'][0][0]['calls']
all_exp['2023-3-17'].loc['options'][0][0]['puts']

# Strike Prices for a Current Expiration Date
all_exp['2023-3-17'].loc['strikes'][0]

# Number of different contracts by Expirations
len(all_exp['2023-3-17'].loc['options'][0][0]['puts'])

86

In [ ]:
# Access to a specific Contract #15
all_exp['2023-3-17'].loc['options'][0][0]['puts'][15]

# Access to a specific Datapoint
all_exp['2023-3-17'].loc['options'][0][0]['puts'][15]['impliedVolatility']

0.392034888305664

# Dataset

## Construction

In [59]:
import os
os.getcwd()

'/Users/ip/Desktop/options-study/VolatilitySurface'

In [60]:
# For each one of the Expiration Dates (Folder)
# Retrieve and save the corresponding chains (Concatenated CALLS & PUTS) - Append a Type Column

for each in dates_human:

    puts = pd.DataFrame.from_records(all_exp[each].loc['options'][0][0]['puts'])
    puts['Type'] = "P"
    calls = pd.DataFrame.from_records(all_exp[each].loc['options'][0][0]['calls'])
    calls['Type'] = "C"
    
    outname = req.headers['date'] + '.csv'
    outdir = './Data/' + ticker + '/' + str(each) + '/'
    
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    puts.append(calls).to_csv(outdir + outname)